In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
# check if notebook is in the right directory if local or get repo if colab

import subprocess

GIT_NAME = 'tiresias_det'
GIT_URL = 'https://github.com/PhileasFrog/tiresias_det.git'

def is_git_repo_with_remote():
    try:
        # check if current direzctory is a git repo
        subprocess.check_output(["git", "rev-parse", "--is-inside-work-tree"])
        # get distant git url
        remote_url = subprocess.check_output(["git", "config", "--get", "remote.origin.url"]).decode("utf-8").strip()
        if remote_url:
            return True, remote_url
        else:
            return False, None
    except subprocess.CalledProcessError:
        return False, None


def setup_notebook_colab(git_name: str, git_url: str):
    # install dependencies: (use cu111 because colab has CUDA 11.1)
    %pip install -U openmim
    !mim install "mmengine>=0.7.0"
    !mim install "mmcv>=2.0.0rc4"
    colab_folder = '/content/' + git_name
    !rm -rf $colab_folder
    print(f"Remove folder {git_name}")
    !git clone $git_url
    print(f"Cloned Git repository from {git_url}")
    %cd $git_name
    print(f"Current directory is {colab_folder}")
    print("Install dependecies")
    %pip install -e .
    print("Download model")
    !mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints
    !mim download mmdet --config yolox_tiny_8x8_300e_coco --dest ./checkpoints
    !mim download mmdet --config ssd512_coco --dest ./checkpoints
    !mim download mmdet --config faster-rcnn_r50_fpn_1x_coco --dest ./checkpoints


def check_notebook_local(git_url: str):
      is_remote, remote_url = is_git_repo_with_remote()
      if not is_remote:
          assert False, 'The current directory is not a remote Git repository.'
      if remote_url.startswith('git@github.com:'):
          remote_url = remote_url.replace('git@github.com:', 'https://github.com/')
      if is_remote and remote_url == git_url:
          print(f"Current folder is OK : {remote_url}")
      else:
          assert False, f'The current directory is not the correct remote Git repository. {remote_url} instead of {git_url}'

def launch_notebook(git_name: str, git_url: str):
    try:
        # clone git if notebook running on colab
        import google.colab
        ON_COLAB = True
        print("You are on Google Colab.")
        setup_notebook_colab(git_name=git_name, git_url=git_url)
    except:
        ON_COLAB = False
        print("You are not on Google Colab. Check if current directory ok")
        check_notebook_local(git_url=git_url)
    return ON_COLAB

ON_COLAB = launch_notebook(git_name=GIT_NAME, git_url=GIT_URL)

You are not on Google Colab. Check if current directory ok
Current folder is OK : https://github.com/PhileasFrog/tiresias_det.git
